In [3]:
import cv2
import pandas
import numpy as np

2.1

In [2]:
for i in range(300, 1100, 2): 
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i) 
    cv2.imshow("I",I) 
    cv2.waitKey(10)
cv2.destroyAllWindows()

2.2

In [ ]:
previous = 0

for i in range (300 ,1100):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    I_gray = I_gray.astype('uint8')
    current = I_gray
    frame = cv2.absdiff(current, previous)

    _ , binary = cv2.threshold(frame, 25, 255, cv2.THRESH_BINARY)

    median = cv2.medianBlur(binary, 3)
    eroded = cv2.erode(median, np.ones((3,3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((3,3), np.uint8))

    cv2.imshow("motion detection", dilated)
    cv2.waitKey (10)
    previous = current
cv2.destroyAllWindows()



Ćwiczenia 2.3

In [ ]:
prev_gray = cv2.imread('pedestrian/input/in%06d.jpg' % 300)
prev_gray = cv2.cvtColor(prev_gray, cv2.COLOR_BGR2GRAY).astype('int')

for i in range(301, 1100):
    current_frame = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    
    current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY).astype('int')
    
    diff = np.abs(current_gray - prev_gray)
    diff_uint8 = diff.astype('uint8')
    
    _, binary = cv2.threshold(diff_uint8, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binary, 3)    
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((15, 15), np.uint8))
    
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated)
    I_VIS = current_frame.copy().astype('uint8') # copy of the input image 
    if (stats.shape[0] > 1): # are there any objects 
        tab = stats[1:,4] # 4 columns without first element 
        pi = np.argmax( tab )# finding the index of the largest item
        pi = pi + 1 # increment because we want the index in stats, not in tab # drawing a bbox
        cv2.rectangle(I_VIS,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi ,1]+stats[pi,3]),(255,0,0),2) # print information about the field and the number of the largest element 
        cv2.putText(I_VIS,"%f" % stats[pi,4],(stats[pi,0],stats[pi,1]),cv2. FONT_HERSHEY_SIMPLEX ,0.5,(255,0,0)) 
    cv2.imshow("Rect", I_VIS)
    
    prev_gray = current_gray.copy()
    
    if cv2.waitKey(10) == 27:
        break

cv2.destroyAllWindows()

Ćwiczenie 2.4

In [ ]:
I1 = cv2.imread('pedestrian/input/in%06d.jpg' % 300)
I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2GRAY)
I1 = I1.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(301, 1100):
    I2 = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
    I2 = I2.astype('int')

    Diff = abs(I2 - I1).astype('uint8')
    _, binarized = cv2.threshold(Diff, 10, 255, cv2.THRESH_BINARY)

    binarized = cv2.medianBlur(binarized, 7)
    binarized = cv2.erode(binarized, np.ones((3, 3), np.uint8))
    binarized = cv2.dilate(binarized, np.ones((3, 3), np.uint8))

    grountruth = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    TP_matrix = np.logical_and((binarized > 0), (grountruth > 0))
    TN_matrix = np.logical_and((binarized == 0), (grountruth == 0))
    FP_matrix = np.logical_and((binarized > 0), (grountruth == 0))
    FN_matrix = np.logical_and((binarized == 0), (grountruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

Precision: 0.8275017557993019
Recall: 0.8199440845166815
F1: 0.823705584718544
